<a href="https://www.kaggle.com/code/yaaangzhou/eda-tutorial-based-on-plotly-express?scriptVersionId=133536823" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/world-most-populated-city-2022-to-2023/World City Populations 2023.csv
/kaggle/input/country-code/country_code.csv


# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
df = pd.read_csv('/kaggle/input/world-most-populated-city-2022-to-2023/World City Populations 2023.csv')

df.drop('Unnamed: 7',axis=1,inplace=True)

country_codes = pd.read_csv("../input/country-code/country_code.csv", index_col=0)

df_all = pd.merge(df,country_codes, how='inner',left_on = 'Country',right_on = 'Country_name')

df = df_all[['Pop2023','Pop2022','City','Country','Continent','rank','code_3digit','growthRate']]

df['TotalPop'] = df['Pop2023']+df['Pop2022']

df.head()

/tmp/ipykernel_20/728524348.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPop'] = df['Pop2023']+df['Pop2022']


,Pop2023,Pop2022,City,Country,Continent,rank,code_3digit,growthRate,TotalPop
0,37194104,37274000,Tokyo,Japan,Asia,1,JPN,-0.0021,74468104
1,19013434,19059856,Osaka,Japan,Asia,10,JPN,-0.0024,38073290
2,9569328,9571596,Nagoya,Japan,Asia,37,JPN,-0.0002,19140924
3,5490271,5502591,Fukuoka,Japan,Asia,74,JPN,-0.0022,10992862
4,2937359,2935436,Shizuoka,Japan,Asia,170,JPN,0.0007,5872795


# Static charts

In [4]:
df_need = df.groupby(['Country','Continent']).sum('TotalPop').reset_index()
df_need

,Country,Continent,Pop2023,Pop2022,rank,growthRate,TotalPop
0,Afghanistan,Asia,4588666,4457882,92,0.0293,9046548
1,Algeria,Africa,3837757,3776261,806,0.0316,7614018
2,Angola,Africa,12748271,12249422,2772,0.2310,24997693
3,Argentina,South America,21863392,21669271,2349,0.0668,43532663
4,Armenia,Asia,1094813,1092028,523,0.0026,2186841
...,...,...,...,...,...,...,...
112,Uruguay,South America,1774396,1767243,295,0.0040,3541639
113,Uzbekistan,Asia,2603243,2573758,196,0.0115,5177001
114,Yemen,Asia,6084281,5865816,2069,0.1590,11950097
115,Zambia,Africa,3944231,3777288,916,0.0832,7721519


# Most and least poputation countries

In [5]:
fig = px.bar(df_need.sort_values('TotalPop', ascending=False).head(20), 
             x='Country', y='TotalPop', 
             color='Continent', title='Top 20 pop cities',
            text='TotalPop', height=700,color_discrete_sequence=px.colors.qualitative.Pastel)

# change style

fig.update_layout(template='plotly_white')

fig.update_layout(xaxis_title='Country', yaxis_title='TotalPop')

fig.update_xaxes(categoryorder='total descending')

fig.show()

## **As we can see, China and India are two countries that have most population in this 2 years. Most of them are from Asia.**

In [6]:
fig = px.bar(df_need.sort_values('TotalPop', ascending=False).tail(20), 
             x='Country', y='TotalPop', 
             color='Continent', title='Least 20 pop cities',
            text='TotalPop', height=700,color_discrete_sequence=px.colors.qualitative.Pastel)

# change style

fig.update_layout(template='plotly_white')

fig.update_layout(xaxis_title='Country', yaxis_title='TotalPop')

fig.update_xaxes(categoryorder='total descending')

fig.show()

## **Here are those countries with least population.**

# Boxplot: Population 

In [7]:
fig = px.box(df[df['Continent']=='Asia'], x="Country", y="TotalPop", color="Country", notched=True,
                title="Box plot of TotalPop by Country in Asia")
fig.update_layout(template='plotly_white')

fig.show()

# Pie plot

In [8]:
df_need = df.groupby(['Country','Continent']).sum('TotalPop').reset_index()

df_need['TotalPop'] = df_need['TotalPop']/df_need['TotalPop'].sum()

df_need['Country'] = np.where(df_need['TotalPop']<0.025,'Others',df_need['Country'])

fig = px.pie(df_need, values='TotalPop', names='Country', title='Population by Country in Asia',
            color_discrete_sequence=px.colors.qualitative.Pastel,labels={'Country':'Country'})
fig.update_traces(textposition='inside', textinfo='label+percent')
fig.update_layout(template='plotly_white')
fig.show()

## **Here, I counted the population share of each country in Asia, and I put the countries with a share less than 0.025 in the OTHER category.**

# Country/City tree map

In [9]:
import plotly.express as px

# Choose top 10 city from each country
top_cities_per_country = df.groupby('Country').apply(lambda x: x.nlargest(10, 'TotalPop')).reset_index(drop=True)

fig = px.treemap(top_cities_per_country, path=['Country', 'City'], values='TotalPop',
                 color='TotalPop', hover_data=['TotalPop'],
                 color_continuous_scale='RdBu', title='Total Salary by College and Player Name')

fig.update_layout(template='plotly_white')

fig.show()


## **A simple visualization: we can see the most populated cities in each country.**

# Geography: Top pop/growthRate by Country

In [10]:
df_need = df.groupby(['code_3digit','Country']).agg({'TotalPop': 'sum', 'growthRate': 'mean'}).reset_index()

import plotly
import plotly.graph_objects as go

pop_map = go.Choropleth(
    locations = df_need['code_3digit'],
    z = df_need['TotalPop'],
    text = df_need['Country'],
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorscale = 'Blues',
    colorbar_title = 'TotalPop'
)

gr_map = go.Choropleth(
    locations = df_need['code_3digit'],
    z = df_need['growthRate'],
    text = df_need['Country'],
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorscale = 'Blues',
    colorbar_title = 'growthRate'
)

data = [pop_map, gr_map]

updatemenus = list([
    dict(type="buttons",
         y=0.7,
         buttons=list([   
            dict(label = 'Population Map',
                 method = 'update',
                 args = [{'visible': [True, False]},
                         {'title': 'Population by Country'}]),

            dict(label = 'GrowthRate Map',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'Average Growth Rate by Country'}])
        ]),
    )
])


layout = dict(updatemenus=updatemenus,
             title_text='Top pop/growthRate by Country',
             geo=dict(
                showframe=False,
                showcoastlines=False,
                projection_type='equirectangular'),
             margin=dict(l=0, r=0, b=0),
             height=500
             )

fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig)

# Animation: Bubble map

In [11]:
fig = px.scatter_geo(df, locations="code_3digit", color="Continent",
                        hover_name="City", size="TotalPop",
                        projection="natural earth", title='Population by City',animation_frame="Continent",)
fig.update_layout(template='plotly')

fig.show()

## **We can see the whole population/average birth rate for each country on the map.**

### **I'm going to update this notebook later. Hope you like this!**